# Monte Carlo Tree Search (MCTS) Testing and Optimization

Este notebook implementa y evalúa el algoritmo MCTS en diferentes juegos.

**Nota importante:** MCTS no se "entrena" como otros algoritmos de ML. Es un algoritmo de búsqueda que construye un árbol desde cero en cada jugada. Lo que haremos es:
- Probar compatibilidad con diferentes juegos
- Evaluar rendimiento con diferentes configuraciones
- Comparar contra otros agentes
- Analizar el impacto de parámetros como número de simulaciones

In [1]:
# Importar librerías y juegos
from games.tictactoe.tictactoe import TicTacToe
from games.nocca_nocca.nocca_nocca import NoccaNocca
from games.kuhn.kuhn import KuhnPoker
from games.leduc.leduc import LeducPoker
from agents.mcts_t import MonteCarloTreeSearch
from agents.agent_random import RandomAgent
from agents.minimax import MiniMax
import numpy as np

# Forzar recarga de módulos para aplicar cambios GPU
import importlib
import sys

if 'agents.mcts_t' in sys.modules:
    importlib.reload(sys.modules['agents.mcts_t'])

# Reimportar la clase actualizada
from agents.mcts_t import MonteCarloTreeSearch

print("MCTS GPU integrado listo")

MCTS GPU integrado listo


## Optimización GPU Integrada

**Cambios implementados**: MCTS ahora incluye aceleración GPU integrada en la clase principal:

1. **Detección automática** de PyTorch MPS/CUDA
2. **Tensores GPU** para recompensas acumuladas  
3. **Rollouts vectorizados** para múltiples simulaciones
4. **Fallback automático** a CPU si GPU no está disponible
5. **Compatibilidad total** con código existente

**Beneficios para MacBook M4**:
- Aprovechamiento de MPS (Metal Performance Shaders)
- Operaciones vectorizadas optimizadas
- Mejor rendimiento en rollouts múltiples

In [2]:
# Verificar configuración GPU y crear agente de prueba
import torch
import time

print("🔍 Verificando configuración GPU...")

# Verificar disponibilidad de PyTorch y MPS
try:
    pytorch_available = True
    print(f"✅ PyTorch disponible: {torch.__version__}")
    print(f"   MPS (Apple Silicon) disponible: {torch.backends.mps.is_available()}")
    print(f"   CUDA disponible: {torch.cuda.is_available()}")
except ImportError:
    pytorch_available = False
    print("⚠️  PyTorch no está instalado - usando CPU")

# Configuración optimizada para MacBook M4
FAST_CONFIG = {
    'episodes_test': 3,        # Reducido para testing rápido
    'episodes_eval': 100,      # Para evaluación
    'simulations_fast': 10,    # Para testing
    'simulations_eval': 25,    # Para evaluación
    'rollouts': 3              # Reducido para velocidad
}

print(f"\n📋 Configuración: {FAST_CONFIG}")

# Crear agente de prueba para verificar GPU
test_game = TicTacToe()
test_game.reset()
print("\n🧪 Creando agente MCTS de prueba...")
test_agent = MonteCarloTreeSearch(test_game, test_game.agents[0], simulations=10, rollouts=3)

# Mostrar información de rendimiento si está disponible
if hasattr(test_agent, 'get_performance_info'):
    print("📊 Info del agente:", test_agent.get_performance_info())
else:
    print("📊 Agente: Versión estándar")

# Función para medir tiempo de ejecución
def time_execution(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - start_time
    print(f"⏱️  Tiempo total: {elapsed:.1f}s")
    return result, elapsed

print("\n✅ Configuración GPU completada!")
print("="*50)

🔍 Verificando configuración GPU...
✅ PyTorch disponible: 2.7.1
   MPS (Apple Silicon) disponible: True
   CUDA disponible: False

📋 Configuración: {'episodes_test': 3, 'episodes_eval': 100, 'simulations_fast': 10, 'simulations_eval': 25, 'rollouts': 3}

🧪 Creando agente MCTS de prueba...
🔧 MCTS usando: MPS (Apple Silicon)
📊 Info del agente: {'device': 'mps', 'gpu_enabled': True, 'torch_available': True, 'simulations': 10, 'rollouts': 3}

✅ Configuración GPU completada!


In [3]:
# Función para mostrar información de los agentes GPU
def show_agent_info():
    """Muestra información sobre las capacidades GPU de los agentes"""
    print("🔍 Información de agentes optimizados:")
    
    # Crear agentes temporales para mostrar info
    temp_game = TicTacToe()
    temp_game.reset()
    
    mcts_agent = MonteCarloTreeSearch(temp_game, temp_game.agents[0], simulations=10, rollouts=3)
    minimax_agent = MiniMax(temp_game, temp_game.agents[1], depth=3)
    
    print("\n📊 MCTS:")
    if hasattr(mcts_agent, 'get_performance_info'):
        print(f"   {mcts_agent.get_performance_info()}")
    else:
        print("   Versión estándar (CPU)")
    
    print("\n📊 MiniMax:")
    if hasattr(minimax_agent, 'get_performance_info'):
        print(f"   {minimax_agent.get_performance_info()}")
    else:
        print("   Versión estándar (CPU)")
    
    return mcts_agent, minimax_agent

# Mostrar información de los agentes
mcts_sample, minimax_sample = show_agent_info()

print("\n🎯 Los algoritmos están usando las versiones optimizadas GPU integradas")
print("   - No se necesitan clases separadas")
print("   - Funcionan exactamente igual que las originales")
print("   - GPU se activa automáticamente si está disponible")
print("="*50)

# Verificar que MCTS tiene GPU integrado
test_game = TicTacToe()
test_agent = MonteCarloTreeSearch(test_game, test_game.agents[0], simulations=10, rollouts=3)

# Mostrar información de GPU
if hasattr(test_agent, 'get_performance_info'):
    info = test_agent.get_performance_info()
    print(f"GPU: {info['gpu_enabled']}, Device: {info['device']}")
else:
    print("Versión estándar (CPU)")

🔍 Información de agentes optimizados:
🔧 MiniMax usando: MPS (Apple Silicon)

📊 MCTS:
   {'device': 'mps', 'gpu_enabled': True, 'torch_available': True, 'simulations': 10, 'rollouts': 3}

📊 MiniMax:
   {'device': 'mps', 'gpu_enabled': True, 'torch_available': True, 'cache_size': 0}

🎯 Los algoritmos están usando las versiones optimizadas GPU integradas
   - No se necesitan clases separadas
   - Funcionan exactamente igual que las originales
   - GPU se activa automáticamente si está disponible
GPU: True, Device: mps


## 1. Pruebas rápidas (10 episodios)

Primero probamos que los agentes funcionen correctamente con pocos episodios.

In [4]:
def test_agent_compatibility(game, agent_class, episodes=None, simulations=None):
    """Prueba la compatibilidad de un agente con un juego"""
    # Usar configuración optimizada si no se especifica
    episodes = episodes or FAST_CONFIG['episodes_test']
    simulations = simulations or FAST_CONFIG['simulations_fast']
    
    results = []
    start_time = time.time()
    errors = 0
    
    for episode in range(episodes):
        game.reset()
        agents = {}
        for agent_id in game.agents:
            agents[agent_id] = agent_class(game=game, agent=agent_id, 
                                         simulations=simulations, 
                                         rollouts=FAST_CONFIG['rollouts'])
        
        step_count = 0
        while not game.game_over() and step_count < 200:
            try:
                action = agents[game.agent_selection].action()
                game.step(action)
                step_count += 1
            except Exception as e:
                errors += 1
                break
        
        episode_results = {agent: game.reward(agent) for agent in game.agents}
        results.append(episode_results)
    
    total_time = time.time() - start_time
    print(f"Episodes: {episodes}, Time: {total_time:.1f}s, Errors: {errors}")
    return results

In [5]:
# Test TicTacToe
tictactoe_game = TicTacToe()
tictactoe_results, time_taken = time_execution(
    test_agent_compatibility, 
    tictactoe_game, 
    MonteCarloTreeSearch
)

Episodes: 3, Time: 0.6s, Errors: 0
⏱️  Tiempo total: 0.6s


In [6]:
# Test Nocca-Nocca
nocca_game = NoccaNocca(max_steps=100, initial_player=0, seed=1)
nocca_results, time_taken = time_execution(
    test_agent_compatibility, 
    nocca_game, 
    MonteCarloTreeSearch
)

Episodes: 3, Time: 34.2s, Errors: 0
⏱️  Tiempo total: 34.2s


In [7]:
# Test Kuhn Poker
kuhn_game = KuhnPoker()
kuhn_results, time_taken = time_execution(
    test_agent_compatibility, 
    kuhn_game, 
    MonteCarloTreeSearch
)

Episodes: 3, Time: 0.1s, Errors: 0
⏱️  Tiempo total: 0.1s


In [8]:
# Test Leduc Poker
leduc_game = LeducPoker()
leduc_results, time_taken = time_execution(
    test_agent_compatibility, 
    leduc_game, 
    MonteCarloTreeSearch
)

Episodes: 3, Time: 0.1s, Errors: 0
⏱️  Tiempo total: 0.1s


## 2. Evaluación de rendimiento (2000 partidas)

Ahora evaluamos el rendimiento de MCTS jugando muchas partidas. 

**Importante:** MCTS no aprende entre partidas - cada juego empieza desde cero. Esta evaluación nos ayuda a entender qué tan bien funciona el algoritmo en cada juego.

In [9]:
import time

def evaluate_mcts_performance(game, episodes=None, simulations=None):
    """Evalúa el rendimiento de MCTS durante muchas partidas"""
    episodes = episodes or FAST_CONFIG['episodes_eval']
    simulations = simulations or FAST_CONFIG['simulations_eval']
    
    win_counts = {agent: 0 for agent in game.agents}
    draw_count = 0
    start_time = time.time()
    errors = 0
    
    for episode in range(episodes):
        game.reset()
        agents = {}
        for agent_id in game.agents:
            agents[agent_id] = MonteCarloTreeSearch(game=game, agent=agent_id, 
                                                  simulations=simulations,
                                                  rollouts=FAST_CONFIG['rollouts'])
        
        step_count = 0
        try:
            while not game.game_over() and step_count < 200:
                action = agents[game.agent_selection].action()
                game.step(action)
                step_count += 1
        except Exception:
            errors += 1
            continue
        
        rewards = {agent: game.reward(agent) for agent in game.agents}
        winner = max(rewards, key=rewards.get) if max(rewards.values()) > 0 else None
        
        if winner:
            win_counts[winner] += 1
        else:
            draw_count += 1
    
    total_time = time.time() - start_time
    print(f"Episodes: {episodes}, Time: {total_time:.1f}s, Wins: {win_counts}, Draws: {draw_count}, Errors: {errors}")
    return win_counts, draw_count

In [10]:
# Evaluar rendimiento en TicTacToe (usando parámetros optimizados M4)
tictactoe_performance, eval_time = time_execution(
    evaluate_mcts_performance, 
    TicTacToe(), 
    episodes=FAST_CONFIG['episodes_eval'], 
    simulations=FAST_CONFIG['simulations_eval']
)

Episodes: 100, Time: 33.3s, Wins: {'X': 71, 'O': 15}, Draws: 14, Errors: 0
⏱️  Tiempo total: 33.3s


In [ ]:
# Evaluar rendimiento en Nocca-Nocca (M4 optimizado)
nocca_performance = evaluate_mcts_performance(
    NoccaNocca(max_steps=100, seed=1), 
    episodes=FAST_CONFIG['episodes_eval'], 
    simulations=FAST_CONFIG['simulations_eval']
)

KeyboardInterrupt: 

Tuve que re-testear mcts para chequear que funcione correctamente en leduc, y nocca nocca demoraba demasiado, asi que lo saltee para esta ultima ejecucion, pero en la notebook de evaluacion final se puede ver que funciona

In [13]:
# Evaluar rendimiento en Kuhn Poker (M4 optimizado)
kuhn_performance = evaluate_mcts_performance(
    KuhnPoker(), 
    episodes=FAST_CONFIG['episodes_eval'], 
    simulations=FAST_CONFIG['simulations_eval']
)

Episodes: 100, Time: 5.9s, Wins: {'agent_0': 50, 'agent_1': 50}, Draws: 0, Errors: 0


In [14]:
# Evaluar rendimiento en Leduc Poker (M4 optimizado)
leduc_performance = evaluate_mcts_performance(
    LeducPoker(), 
    episodes=FAST_CONFIG['episodes_eval'], 
    simulations=FAST_CONFIG['simulations_eval']
)

Episodes: 100, Time: 161.4s, Wins: {'agent_0': 23, 'agent_1': 46}, Draws: 31, Errors: 0


## 3. Optimización de parámetros

MCTS tiene parámetros importantes que afectan su rendimiento:
- **Número de simulaciones**: Más simulaciones = mejor calidad de decisión pero más tiempo
- **Número de rollouts**: Afecta la calidad de la evaluación de nodos
- **Constante UCB**: Controla exploración vs explotación

Probemos diferentes configuraciones para encontrar un buen balance.

In [15]:
def test_mcts_configurations(game, configs, episodes=None):
    """Prueba diferentes configuraciones de MCTS para encontrar la óptima"""
    episodes = episodes or FAST_CONFIG['episodes_test'] * 3
    
    results = {}
    
    for config_name, params in configs.items():
        win_counts = {agent: 0 for agent in game.agents}
        draw_count = 0
        errors = 0
        start_time = time.time()
        
        for episode in range(episodes):
            game.reset()
            agents = {}
            for agent_id in game.agents:
                agents[agent_id] = MonteCarloTreeSearch(game=game, agent=agent_id, **params)
            
            step_count = 0
            try:
                while not game.game_over() and step_count < 200:
                    action = agents[game.agent_selection].action()
                    game.step(action)
                    step_count += 1
            except Exception:
                errors += 1
                continue
            
            rewards = {agent: game.reward(agent) for agent in game.agents}
            winner = max(rewards, key=rewards.get) if max(rewards.values()) > 0 else None
            
            if winner:
                win_counts[winner] += 1
            else:
                draw_count += 1
        
        elapsed = time.time() - start_time
        results[config_name] = {'wins': win_counts, 'draws': draw_count, 'time': elapsed, 'errors': errors}
        print(f"{config_name}: Time={elapsed:.1f}s, Wins={win_counts}, Draws={draw_count}, Errors={errors}")
    
    return results

# Configuraciones para M4
mcts_configs = {
    'Ultra_Fast': {'simulations': 5, 'rollouts': 2},
    'Fast': {'simulations': 10, 'rollouts': 3},
    'Balanced': {'simulations': 25, 'rollouts': 5},
    'Quality': {'simulations': 50, 'rollouts': 8}
}

# Probar en TicTacToe
tictactoe_configs = test_mcts_configurations(TicTacToe(), mcts_configs)

Ultra_Fast: Time=0.4s, Wins={'X': 7, 'O': 2}, Draws=0, Errors=0
Fast: Time=0.9s, Wins={'X': 7, 'O': 1}, Draws=1, Errors=0
Fast: Time=0.9s, Wins={'X': 7, 'O': 1}, Draws=1, Errors=0
Balanced: Time=7.7s, Wins={'X': 9, 'O': 0}, Draws=0, Errors=0
Balanced: Time=7.7s, Wins={'X': 9, 'O': 0}, Draws=0, Errors=0
Quality: Time=21.3s, Wins={'X': 0, 'O': 0}, Draws=9, Errors=0
Quality: Time=21.3s, Wins={'X': 0, 'O': 0}, Draws=9, Errors=0


In [16]:
def evaluate_agents(game, agent1_class, agent2_class, episodes=None, **kwargs):
    """Evalúa dos agentes uno contra el otro"""
    episodes = episodes or FAST_CONFIG['episodes_test'] * 5
    
    def get_agent_kwargs(agent_class, all_kwargs):
        """Filtra los kwargs apropiados para cada tipo de agente"""
        if agent_class.__name__ == 'RandomAgent':
            # RandomAgent solo necesita game y agent
            return {}
        elif agent_class.__name__ == 'MonteCarloTreeSearch':
            # MCTS necesita simulations y rollouts
            return {k: v for k, v in all_kwargs.items() if k in ['simulations', 'rollouts']}
        elif agent_class.__name__ == 'MiniMax':
            # MiniMax necesita depth
            return {k: v for k, v in all_kwargs.items() if k in ['depth']}
        else:
            # Para otros agentes, pasar todos los kwargs
            return all_kwargs
    
    agent1_wins = 0
    agent2_wins = 0
    draws = 0
    errors = 0
    start_time = time.time()
    
    for episode in range(episodes):
        game.reset()
        
        # Obtener kwargs apropiados para cada agente
        agent1_kwargs = get_agent_kwargs(agent1_class, kwargs)
        agent2_kwargs = get_agent_kwargs(agent2_class, kwargs)
        
        # Alternar qué agente juega primero
        if episode % 2 == 0:
            agents = {
                game.agents[0]: agent1_class(game=game, agent=game.agents[0], **agent1_kwargs),
                game.agents[1]: agent2_class(game=game, agent=game.agents[1], **agent2_kwargs)
            }
        else:
            agents = {
                game.agents[0]: agent2_class(game=game, agent=game.agents[0], **agent2_kwargs),
                game.agents[1]: agent1_class(game=game, agent=game.agents[1], **agent1_kwargs)
            }
        
        step_count = 0
        try:
            while not game.game_over() and step_count < 200:
                action = agents[game.agent_selection].action()
                game.step(action)
                step_count += 1
        except Exception:
            errors += 1
            continue
        
        # Determinar ganador
        rewards = {agent: game.reward(agent) for agent in game.agents}
        
        if episode % 2 == 0:  # agent1 jugó primero
            if rewards[game.agents[0]] > rewards[game.agents[1]]:
                agent1_wins += 1
            elif rewards[game.agents[1]] > rewards[game.agents[0]]:
                agent2_wins += 1
            else:
                draws += 1
        else:  # agent2 jugó primero
            if rewards[game.agents[0]] > rewards[game.agents[1]]:
                agent2_wins += 1
            elif rewards[game.agents[1]] > rewards[game.agents[0]]:
                agent1_wins += 1
            else:
                draws += 1
    
    elapsed = time.time() - start_time
    print(f"{agent1_class.__name__} {agent1_wins} - {agent2_wins} {agent2_class.__name__} | Empates: {draws} | Errores: {errors}")
    return agent1_wins, agent2_wins, draws

In [17]:
# MCTS vs Random en TicTacToe (M4 optimizado)
mcts_vs_random_ttt = evaluate_agents(
    TicTacToe(), 
    MonteCarloTreeSearch, 
    RandomAgent, 
    simulations=FAST_CONFIG['simulations_eval'],
    rollouts=FAST_CONFIG['rollouts']
)

MonteCarloTreeSearch 14 - 1 RandomAgent | Empates: 0 | Errores: 0


In [18]:
# MCTS vs MiniMax en TicTacToe (M4 optimizado)
mcts_vs_minimax_ttt = evaluate_agents(
    TicTacToe(), 
    MonteCarloTreeSearch, 
    MiniMax, 
    simulations=FAST_CONFIG['simulations_eval'],
    rollouts=FAST_CONFIG['rollouts'],
    depth=3
)

MonteCarloTreeSearch 3 - 4 MiniMax | Empates: 8 | Errores: 0


In [19]:
# MCTS vs Random en Nocca-Nocca (M4 optimizado)
mcts_vs_random_nocca = evaluate_agents(
    NoccaNocca(max_steps=50, seed=1), 
    MonteCarloTreeSearch, 
    RandomAgent, 
    simulations=FAST_CONFIG['simulations_fast'],
    rollouts=FAST_CONFIG['rollouts']
)

KeyboardInterrupt: 

Aca tambien lo corte para la ultima ejecucion, pero me quede con el ultimo resultado y esta acontinuacion:

MonteCarloTreeSearch 7 - 2 RandomAgent | Empates: 6 | Errores: 0

In [20]:
# MCTS vs Random en Leduc Poker (M4 optimizado)
mcts_vs_random_leduc = evaluate_agents(
    LeducPoker(), 
    MonteCarloTreeSearch, 
    RandomAgent, 
    simulations=FAST_CONFIG['simulations_fast'],
    rollouts=FAST_CONFIG['rollouts']
)

MonteCarloTreeSearch 14 - 1 RandomAgent | Empates: 0 | Errores: 0


## 5. Análisis de resultados

Resumen de resultados y análisis de performance de MCTS.

In [21]:
print("=== MCTS PERFORMANCE ANALYSIS ===")
print(f"Config: {FAST_CONFIG}")
print(f"TicTacToe: {len(tictactoe_results)} episodes")
print(f"Nocca-Nocca: {len(nocca_results)} episodes") 
print(f"Kuhn Poker: {len(kuhn_results)} episodes")
print(f"Leduc Poker: {len(leduc_results)} episodes")

if 'tictactoe_performance' in locals():
    print(f"TicTacToe perf: {tictactoe_performance}")
if 'nocca_performance' in locals():
    print(f"Nocca-Nocca perf: {nocca_performance}")
if 'kuhn_performance' in locals():
    print(f"Kuhn Poker perf: {kuhn_performance}")
if 'leduc_performance' in locals():
    print(f"Leduc Poker perf: {leduc_performance}")

if 'tictactoe_configs' in locals():
    for config, results in tictactoe_configs.items():
        print(f"{config}: Time={results['time']:.1f}s, Wins={results['wins']}")

if 'mcts_vs_random_ttt' in locals():
    print(f"MCTS vs Random (TicTacToe): {mcts_vs_random_ttt}")
if 'mcts_vs_minimax_ttt' in locals():
    print(f"MCTS vs MiniMax (TicTacToe): {mcts_vs_minimax_ttt}")
if 'mcts_vs_random_nocca' in locals():
    print(f"MCTS vs Random (Nocca-Nocca): {mcts_vs_random_nocca}")
if 'mcts_vs_random_leduc' in locals():
    print(f"MCTS vs Random (Leduc Poker): {mcts_vs_random_leduc}")

=== MCTS PERFORMANCE ANALYSIS ===
Config: {'episodes_test': 3, 'episodes_eval': 100, 'simulations_fast': 10, 'simulations_eval': 25, 'rollouts': 3}
TicTacToe: 3 episodes
Nocca-Nocca: 3 episodes
Kuhn Poker: 3 episodes
Leduc Poker: 3 episodes
TicTacToe perf: ({'X': 71, 'O': 15}, 14)
Kuhn Poker perf: ({'agent_0': 50, 'agent_1': 50}, 0)
Leduc Poker perf: ({'agent_0': 23, 'agent_1': 46}, 31)
Ultra_Fast: Time=0.4s, Wins={'X': 7, 'O': 2}
Fast: Time=0.9s, Wins={'X': 7, 'O': 1}
Balanced: Time=7.7s, Wins={'X': 9, 'O': 0}
Quality: Time=21.3s, Wins={'X': 0, 'O': 0}
MCTS vs Random (TicTacToe): (14, 1, 0)
MCTS vs MiniMax (TicTacToe): (3, 4, 8)
MCTS vs Random (Leduc Poker): (14, 1, 0)


## Análisis de Resultados

**Configuración utilizada:** `episodes_test=3`, `episodes_eval=100`, `simulations_fast=10`, `simulations_eval=25`, `rollouts=3`.

### 1. Desempeño de MCTS en cada juego (evaluación)

- **TicTacToe:** X ganó 76 partidas (76%), O ganó 15 partidas (15%), 9 empates (9%).
- **Nocca-Nocca:** White ganó 98 partidas (98%), Black ganó 2 partidas (2%), 0 empates.
- **Kuhn Poker:** Ventaja ligera de `agent_0` con 52 victorias (52%) vs `agent_1` con 48 victorias (48%), 0 empates.
- **Leduc Poker:** [Resultados se mostrarán después de ejecutar las celdas correspondientes]

### 2. Optimización de parámetros

- **Ultra_Fast** `(simulations=5, rollouts=2)`: Tiempo=0.4s, X=3 (50%), O=3 (50%).
- **Fast** `(simulations=10, rollouts=3)`: Tiempo=1.0s, X=5 (71%), O=2 (29%).
- **Balanced** `(simulations=25, rollouts=5)`: Tiempo=7.9s, X=9 (100%), O=0 (0%).
- **Quality** `(simulations=50, rollouts=8)`: Tiempo=23.3s, X=0, O=0 (posible timeout o empates).

### 3. Comparativa con otros agentes en TicTacToe, Nocca-Nocca y Leduc Poker

- **MCTS vs Random (TicTacToe):** MCTS ganó 13 partidas (87%), Random 2 (13%), 0 empates.
- **MCTS vs MiniMax (TicTacToe):** MiniMax domina con 9 victorias vs 0 de MCTS y 6 empates.
- **MCTS vs Random (Nocca-Nocca):** MCTS ganó 7 (54%), Random 2 (15%), 6 empates (31%).
- **MCTS vs Random (Leduc Poker):** [Resultados se mostrarán después de ejecutar las celdas correspondientes]

**Conclusiones:**

- MCTS con configuraciones moderadas (e.g., Fast o Balanced) ofrece un buen compromiso entre velocidad y rendimiento en TicTacToe.
- Para juegos determinísticos con un espacio de estados reducido (TicTacToe), los algoritmos clásicos como Minimax pueden superar a MCTS en profundidad limitada.
- En juegos con mayor aleatoriedad o complejidad (Kuhn Poker, Nocca-Nocca, Leduc Poker), MCTS muestra un comportamiento estable comparado con agentes aleatorios.
- Leduc Poker, siendo más complejo que Kuhn Poker, debería proporcionar un mejor benchmark para evaluar las capacidades de MCTS en juegos de información imperfecta.